In [ ]:
## For colab runs
# from google.colab import drive
# drive.mount('/content/gdrive')

# !mkdir -p data
# !cp /content/gdrive/My\ Drive/uploads/a9a.2 ./data # load this data to the specified dir on your drive first

# DGM Minimal Environment 

TODO: Add to `dummy_consensus_variation` check of a graph connectedness

In [ ]:
import torch
import pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_svmlight_file
from src.objectives import (
    LeastSquares, LogRegression,
    StochLeastSquares, StochLogRegression)
from src.methods import EXTRA, DIGing, DAccGD, DSGD
from src.utils import (
    consensus_variation,
    PythonGraph)

# from src.sparse import objectives as obj

In [ ]:
num_nodes = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
A, b = load_svmlight_file('data/a9a')
A = torch.Tensor(A.todense()).to(device)
b = torch.Tensor(b).to(device)

In [ ]:
# For simulating a graph evolution,
# only graphs like 'erdos_renyi' is appropriate

p = .3
graph = 'random_geometric'
# graph = 'erdos_renyi'
# graph = 'path'
# graph = 'cycle'
# graph = 'complete'

F = LogRegression(A, b, num_nodes)
Fs = StochLogRegression(A, b, num_nodes,)
X0 = torch.zeros(num_nodes, A.size(1)).to(device)
gen = PythonGraph(F, graph, p).gen
# gen = lambda: dummy_consensus_variation(F, graph, p)
# gen = lambda: (None, consensus_variation(F, p))

opt1 = EXTRA(F, gen, alpha=1.)
opt2 = DIGing(F, gen, alpha=1.)

consensus_iters = 10
opt3 = DAccGD(F, gen, L=.2, mu=0., T=consensus_iters)

gen_ = lambda : gen()[1]
opt4 = DSGD(Fs, gen_, alpha=4.)

In [ ]:
%%time
n_iters = 500
opt1.run(X0, n_iters=n_iters);
opt2.run(X0, n_iters=n_iters);
opt3.run(X0, n_iters=n_iters // consensus_iters);
# opt4.run(X0, n_iters=n_iters);

opt3.logs['i'] = [item * consensus_iters for item in opt3.logs['i']]

for opt in [opt1, opt2, opt3]:
    for key in ['i', 'fn', 'dist2con']:
        opt.logs[key] = np.array(opt.logs[key])

In [ ]:
with open('./logreg_solutions/a9a/regcoef=0.00e+00', 'rb') as file:
    f_star = pickle.load(file)['func_star']

In [ ]:
hist_len = 500

plt.figure(figsize=(8, 8))
plt.plot(opt1.logs['fn'][:hist_len] - f_star, label='EXTRA', marker='o', markevery=50)
plt.plot(opt2.logs['fn'][:hist_len] - f_star, label='DIGing', marker='v', markevery=(25, 50))
plt.plot(opt3.logs['i'][:hist_len // consensus_iters], 
         opt3.logs['fn'][:hist_len // consensus_iters] - f_star, 
         marker='x', markevery=10, label='DAccGD')
# plt.plot(opt4.logs['fn'] - f_star, label='DSGD', marker='^', markevery=100)
# plt.title('Optimization Functional Value over Iteration Number', size=20)
plt.xlabel('communication steps', size=20)
plt.ylabel(r'$f(\overline{x}_k) - f^*$', size=20)
plt.yscale('log')
plt.grid()
plt.legend();

# plt.savefig('../Decentralized + inexactness/en/figures/a9a_residual.png')

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(opt1.logs['dist2con'][:hist_len], label='EXTRA', marker='o', markevery=50)
plt.plot(opt2.logs['dist2con'][:hist_len], label='DIGing', marker='v', markevery=(25, 50))
plt.plot(opt3.logs['i'][:hist_len // consensus_iters], 
         opt3.logs['dist2con'][:hist_len // consensus_iters], 
         label='DAccGD', marker='x', markevery=10)
# plt.plot(opt4.logs['dist2con'], label='DGSGD', marker='^', markevery=50)
plt.yscale('log')
plt.ylabel(r'$||(I-\frac{1}{n}11^T)X||^2$', size=20)
plt.xlabel('communication steps', size=20)
plt.grid()
plt.legend();

# plt.savefig('../Decentralized + inexactness/en/figures/a9a_consensus.png')